### This notebook only works inside the github.com/cajal/microns-proximities environment and is included for documentation purposes

In [ ]:
%%capture
import numpy as np
import pandas as pd
from microns_proximities import minnie_proximities as mpx
import shutil

In [ ]:
axon_nuc_ids = [327859, 560530, 222998]
dend_nuc_ids = [587885, 292965, 587744]

In [ ]:
# navigate datajoint database
skeleton_prc_set_rel = mpx.SkeletonProcessedSet & (mpx.ProximitySynapseSet.Info & mpx.ProximitySynapseSetRecommended)
skeleton_prc_axon_rel = mpx.SkeletonProcessed.r1p([{'nucleus_id': n} for n in axon_nuc_ids]) & (mpx.SkeletonProcessedSet.Proximity & skeleton_prc_set_rel)
skeleton_prc_dend_rel = mpx.SkeletonProcessed.r1p([{'nucleus_id': n} for n in dend_nuc_ids]) & (mpx.SkeletonProcessedSet.Proximity & skeleton_prc_set_rel)
skeleton_axon_rel = mpx.Skeleton.r1p(skeleton_prc_axon_rel.proj())
skeleton_dend_rel = mpx.Skeleton.r1p(skeleton_prc_dend_rel.proj())

# make data lookup df
axon_df = pd.DataFrame(skeleton_axon_rel.fetch('KEY', as_dict=True))
dend_df = pd.DataFrame(skeleton_dend_rel.fetch('KEY', as_dict=True))
axon_df['compartment'] = 'axon'
dend_df['compartment'] = 'dend'

# get source filepaths
axon_src_dict, _ = mpx.Skeleton.get_filepath(axon_df, source='MeshworkAxonDendrite', compartment='axon')
_, dend_src_dict = mpx.Skeleton.get_filepath(dend_df, source='AutoProofreadNeuron', compartment='dendrite')

# prepare skeleton lookup df
df = pd.concat([axon_df, dend_df])
df['filename'] = df.apply(lambda r: f"skeleton_{r.compartment}_{r.nucleus_id}.npz", axis=1)

In [ ]:
# copy and rename skeleton files
for row in df.itertuples():
    if row.compartment == 'axon':
        shutil.copy(axon_src_dict[row.skeleton_id], row.filename)
    elif row.compartment == 'dend':
        shutil.copy(dend_src_dict[row.skeleton_id], row.filename)

df.to_csv('skeleton_lookup.csv', index=False) # save skeleton info dataframe as csv